In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline
from sklearn.linear_model import LinearRegression,LogisticRegressionCV
from sklearn import  metrics
from sklearn. model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.ensemble import ExtraTreesClassifier
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import seaborn as sns
from scipy import stats
%matplotlib inline
import os



In [ ]:
import pandas as pd

application_test = pd.read_csv("../input/home-credit-default-risk/application_test.csv")
application_train = pd.read_csv("../input/home-credit-default-risk/application_train.csv")
application_test['source']='test'
application_train['source']='train'
data=pd.concat([application_train,application_test],axis=0)
print(application_train.shape,application_test.shape)
print(data.shape)
data.head()

In [ ]:
vars_with_na=[var for var in data.columns if data[var].isnull().sum()>1]
vars_with_high_missing_range=[var for var in data.columns if np.round(data[var].isnull().mean(),3)>0.4]
for var in vars_with_na:
    print(var,np.round(application_train[var].isnull().mean(),3),"% missing values")


for var in vars_with_na:
    print("================"+ var +"===================" )
    print(data[var].value_counts())
    print()
    
data.dtypes.value_counts()


In [ ]:
data=data.drop(vars_with_high_missing_range,axis=1)

In [ ]:
remainingvars_with_na=[var for var in data.columns if data[var].isnull().sum()>1]
for var in remainingvars_with_na:
    print(var,np.round(data[var].isnull().mean(),3),"% missing values")

In [ ]:


discrete_vars_from_remainingvars = [var for var in remainingvars_with_na if len(data[var].unique())<20 and var not in ['SK_ID_CURR','TARGET']]
print(discrete_vars_from_remainingvars)

remaining=[i for i in remainingvars_with_na if i not in discrete_vars_from_remainingvars]

num_vars_from_remainingvars=[var for var in remaining if data[var].dtypes!='O' and var not in ['TARGET']]
print(num_vars_from_remainingvars)


In [ ]:

for var in num_vars_from_remainingvars:
    data[var]=data[var].fillna(data[var].mean())


for var in discrete_vars_from_remainingvars:
    data[var]=data[var].fillna(data[var].mode()[0])

data['DAYS_LAST_PHONE_CHANGE']=data['DAYS_LAST_PHONE_CHANGE'].fillna(data[var].mode()[0])

In [ ]:
num_columns=[var for var in data.columns if (data[var].dtypes=='float64') and var not in ['SK_ID_CURR','TARGET','source']]


In [ ]:

#del application_train,application_test

from sklearn.preprocessing import StandardScaler
sd=StandardScaler()
for var in num_columns:
    data[[var]]=sd.fit_transform(data[[var]])
    


In [ ]:
obj_cols = [col for col in list(data.columns) if data[col].dtype == 'object' and col not in ['TARGET','SK_ID_CURR','source']]

for col in obj_cols:
        data = pd.concat([data, pd.get_dummies(data[col], prefix=col,drop_first=True)], axis=1)

In [ ]:
data=data.drop(obj_cols,axis=1)

In [ ]:
data.shape

In [ ]:
for var in data.columns:
    print("================"+ var +"===================" )
    print(data[var].value_counts())
    print()


In [ ]:
data.to_pickle("./preprocessed.pkl")

In [ ]:
import os 
dir_path = os.path.dirname(os.path.realpath("preprocessed.pkl"))
print(dir_path)

In [ ]:
#print(data.isnull().sum())
train_data=data[data['source']=='train']
test_data=data[data['source']=='test']
print(train_data.shape,test_data.shape)
train_data=train_data.drop('source',axis=1)
test_data=test_data.drop('source',axis=1)


x_train_data=train_data.drop('TARGET',axis=1)
y_train_data=train_data[['TARGET']]
print(x_train_data.shape,y_train_data.shape)

x_test_data=test_data.drop('TARGET',axis=1)
y_test_data=test_data[['TARGET']]

print(x_test_data.shape,y_test_data.shape)

In [ ]:
from numpy.linalg import svd
from sklearn.decomposition import TruncatedSVD

svd_tr = TruncatedSVD(n_components=40)
x_tsvd=svd_tr.fit_transform(x_train_data)
x_tsvd = pd.DataFrame(x_tsvd)
x_tsvd.head()

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=40)
x_pca = pca.fit_transform(x_train_data)
x_pca = pd.DataFrame(x_pca)
x_pca.head()

In [ ]:
from sklearn.decomposition import PCA
help(PCA())

In [ ]:
classifiers = [
    KNeighborsClassifier(3),
    GaussianNB(),
    LogisticRegression(),
    SVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier()]


for clf in classifiers:
    
    clf.fit(x_train_data, y_train_data)
    
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    
    y_pred = clf.predict(x_test_data)
    
    print(y_pred)
    #print("Accuracy:     ", round(metrics.accuracy_score(y_test_data, y_pred),4)*100)
    #print("Precision:    ", round(metrics.precision_score(y_test_data, y_pred),4)*100)
    #print("Recall:       ", round(metrics.recall_score(y_test_data, y_pred),4)*100)

In [ ]:
obj_cols

In [ ]:
def analyse_na_value(df, var):
    df = df.copy()
    
    # let's make a variable that indicates 1 if the observation was missing or zero otherwise
    df[var] = np.where(df[var].isnull(), 1, 0)
    
    # let's calculate the mean SalePrice where the information is missing or present
    df.groupby('TARGET')[var].plot.bar()
    plt.title(var)
    plt.show()

def analyse_barplot(df,column_name):
    figure=plt.figure(figsize=[9,8])
    sns.barplot(df[column_name].value_counts().index,df[column_name].value_counts())
    plt.xlabel(column_name)
    plt.plot()


for var in vars_with_na:
    analyse_barplot(application_train, var)

In [ ]:
classifiers = [
    KNeighborsClassifier(3),
    GaussianNB(),
    LogisticRegression(),
    SVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    GradientBoostingClassifier()]


for clf in classifiers:
    
    clf.fit(x_train_data, y_train_data)
    
    name = clf.__class__.__name__
    
    print("="*30)
    print(name)
    
    print('****Results****')
    
    y_pred = clf.predict(train_x)
    
    print("Accuracy:     ", round(metrics.accuracy_score(train_y, y_pred),4)*100)
    print("Precision:    ", round(metrics.precision_score(train_y, y_pred),4)*100)
    print("Recall:       ", round(metrics.recall_score(train_y, y_pred),4)*100)

In [ ]:
from IPython.display import FileLink
FileLink(r'preprocessed.pkl')